In [1]:
import os
import pathlib
import urllib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [2]:
tf.keras.utils.get_file(
    'speech.zip',
    origin='http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip',
    extract=True,
    cache_dir='.', cache_subdir='data'
)

'.\\data\\speech.zip'

In [3]:
pwd

'D:\\leeky_ai'

In [4]:
class_names = os.listdir('D:\leeky_ai\data\mini_speech_commands')
# class_names != 'README.md'
class_names = [w for w in class_names if w != 'README.md']

In [5]:
class_names

['down', 'go', 'left', 'no', 'right', 'stop', 'up', 'yes']

In [6]:
import glob

In [19]:
# pathlib.Path('D:\leeky_ai\data\mini_speech_commands')

WindowsPath('D:/leeky_ai/data/mini_speech_commands')

In [7]:
filenames = glob.glob('D:\leeky_ai\data\mini_speech_commands'+'/*/*')

In [8]:
# 마지막 10
filenames[-10:]

['D:\\leeky_ai\\data\\mini_speech_commands\\yes\\fd395b74_nohash_1.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\yes\\fd395b74_nohash_2.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\yes\\fd395b74_nohash_3.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\yes\\fde2dee7_nohash_0.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\yes\\fde2dee7_nohash_1.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\yes\\fe1916ba_nohash_1.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\yes\\fe5c4a7a_nohash_0.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\yes\\ff21fb59_nohash_0.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\yes\\ffd2ba2f_nohash_0.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\yes\\ffd2ba2f_nohash_2.wav']

In [10]:
# 앞에서 10
filenames[:10]

['D:\\leeky_ai\\data\\mini_speech_commands\\down\\004ae714_nohash_0.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\down\\00b01445_nohash_1.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\down\\00f0204f_nohash_0.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\down\\0132a06d_nohash_1.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\down\\0132a06d_nohash_4.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\down\\0137b3f4_nohash_2.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\down\\014f9f65_nohash_0.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\down\\016e2c6d_nohash_0.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\down\\016e2c6d_nohash_2.wav',
 'D:\\leeky_ai\\data\\mini_speech_commands\\down\\01b4757a_nohash_0.wav']

In [12]:
filenames =  tf.random.shuffle(filenames)
filenames[:10]

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'D:\\leeky_ai\\data\\mini_speech_commands\\up\\b7a6f709_nohash_0.wav',
       b'D:\\leeky_ai\\data\\mini_speech_commands\\go\\e7ea8b76_nohash_7.wav',
       b'D:\\leeky_ai\\data\\mini_speech_commands\\up\\f33660af_nohash_1.wav',
       b'D:\\leeky_ai\\data\\mini_speech_commands\\go\\726382d6_nohash_0.wav',
       b'D:\\leeky_ai\\data\\mini_speech_commands\\down\\462e1919_nohash_0.wav',
       b'D:\\leeky_ai\\data\\mini_speech_commands\\left\\24ed94ab_nohash_0.wav',
       b'D:\\leeky_ai\\data\\mini_speech_commands\\up\\bbb2eb5b_nohash_1.wav',
       b'D:\\leeky_ai\\data\\mini_speech_commands\\left\\8dd24423_nohash_1.wav',
       b'D:\\leeky_ai\\data\\mini_speech_commands\\no\\b97c9f77_nohash_2.wav',
       b'D:\\leeky_ai\\data\\mini_speech_commands\\yes\\2275edbb_nohash_0.wav'],
      dtype=object)>

In [15]:
dir_path = 'D:\leeky_ai\data\mini_speech_commands'
for dirname in class_names:
    target = dir_path + f"/{dirname}"
    print(f"{dirname}'s count =  {len(os.listdir(target))}" )

down's count =  1000
go's count =  1000
left's count =  1000
no's count =  1000
right's count =  1000
stop's count =  1000
up's count =  1000
yes's count =  1000


In [29]:
# 데이터 분할  8:1:1로 분할  훈련 검증 테스트
split8 = int(len(filenames)*0.8)
split1 = int((len(filenames)-split8)/2)
train_files = filenames[: split8 ]
val_files = filenames[split8: split8+split1]
test_files = filenames[-split1: ]

In [31]:
len(train_files),len(val_files),len(test_files), len(filenames)

(6400, 800, 800, 8000)

In [34]:
# 오디오 파일을 수치 데이터로 변경하기
# 오디오 초당 셈플링 --> 시계열 데이터 -- LSTM
file = tf.io.read_file('D:/leeky_ai/data/mini_speech_commands/down/00b01445_nohash_1.wav')
audio, sample_rate = tf.audio.decode_wav(contents=file)                

In [44]:
audio.shape

TensorShape([16000, 1])

In [45]:
tf.squeeze(audio).shape

TensorShape([16000])

In [46]:
# 디렉토리명을 레이블정보로 취득